In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#This is for installing category encoders since that is used later on in the code. Just uncomment and run 
# pip install category_encoders

In [3]:
#Let this cell be used for AWS connection
# import boto3, re, sys, math, json, os, sagemaker, urllib.request
# from sagemaker import get_execution_role
# from sagemaker.predictor import csv_serializer

#This is for AWS connection, it is what i used for HW4 to connect to Sagemaker.


# # Define IAM role
# role = get_execution_role()
# prefix = 'sagemaker/DEMO-xgboost-dm'
# my_region = boto3.session.Session().region_name # set the region of the instance

# # this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

# print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

# bucket_name = 'INSERT BUCKET NAME HERE' 
# s3 = boto3.resource('s3')
# try:
#     if  my_region == 'us-east-1':
#       s3.create_bucket(Bucket=bucket_name)
#     else: 
#       s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
#     print('S3 bucket created successfully')
# except Exception as e:
#     print('S3 error: ',e)
    
# try:
#   data = pd.read_csv('INSERT PATH TO THE CSV FILE FROM S3',index_col=0)
#   print('Success: Data loaded into dataframe.')
# except Exception as e:
#     print('Data load error: ',e)

In [4]:
#Use this cell to connect to Google Cloud. I don't have any code here so you will have to write that. 

In [5]:
#Importing data from local. Comment this out for Cloud Connection
data = pd.read_csv("weatherAUS.csv")
# data.head()

In [6]:
#You should not have to change anything from this cell down. 
data = data[data['RainTomorrow'].notna()]

In [7]:
# col = data.columns
# print(col)

y = data['RainTomorrow']
X = data.drop(['RainTomorrow'], axis = 1)


In [8]:
# split X and y into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [9]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape

((113754, 22), (28439, 22))

In [10]:
X_train.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
dtype: object

In [11]:
# display categorical variables

categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']

categorical

['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']

In [12]:
# display numerical variables

numerical = [col for col in X_train.columns if X_train[col].dtypes != 'O']

numerical

['MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustSpeed',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm']

In [13]:
# check missing values in numerical variables in X_train

X_train[numerical].isnull().sum()

MinTemp            495
MaxTemp            264
Rainfall          1139
Evaporation      48718
Sunshine         54314
WindGustSpeed     7367
WindSpeed9am      1086
WindSpeed3pm      2094
Humidity9am       1449
Humidity3pm       2890
Pressure9am      11212
Pressure3pm      11186
Cloud9am         43137
Cloud3pm         45768
Temp9am            740
Temp3pm           2171
dtype: int64

In [14]:
# check missing values in numerical variables in X_test

X_test[numerical].isnull().sum()

MinTemp            142
MaxTemp             58
Rainfall           267
Evaporation      12125
Sunshine         13502
WindGustSpeed     1903
WindSpeed9am       262
WindSpeed3pm       536
Humidity9am        325
Humidity3pm        720
Pressure9am       2802
Pressure3pm       2795
Cloud9am         10520
Cloud3pm         11326
Temp9am            164
Temp3pm            555
dtype: int64

In [15]:
# impute missing values in X_train and X_test with respective column median in X_train

for df1 in [X_train, X_test]:
    for col in numerical:
        col_median=X_train[col].median()
        df1[col].fillna(col_median, inplace=True)  

C:\Users\bacch\Anaconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [16]:
# check again missing values in numerical variables in X_train

X_train[numerical].isnull().sum()

MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustSpeed    0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
dtype: int64

In [17]:
# check missing values in numerical variables in X_test

X_test[numerical].isnull().sum()

MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustSpeed    0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
dtype: int64

In [18]:
# print percentage of missing values in the categorical variables in training set

X_train[categorical].isnull().mean()

Date           0.000000
Location       0.000000
WindGustDir    0.065114
WindDir9am     0.070134
WindDir3pm     0.026443
RainToday      0.010013
dtype: float64

In [19]:
# print categorical variables with missing data

for col in categorical:
    if X_train[col].isnull().mean()>0:
        print(col, (X_train[col].isnull().mean()))

WindGustDir 0.06511419378659213
WindDir9am 0.07013379749283542
WindDir3pm 0.026443026179299188
RainToday 0.01001283471350458


In [20]:
# impute missing categorical variables with most frequent value

for df2 in [X_train, X_test]:
    df2['WindGustDir'].fillna(X_train['WindGustDir'].mode()[0], inplace=True)
    df2['WindDir9am'].fillna(X_train['WindDir9am'].mode()[0], inplace=True)
    df2['WindDir3pm'].fillna(X_train['WindDir3pm'].mode()[0], inplace=True)
    df2['RainToday'].fillna(X_train['RainToday'].mode()[0], inplace=True)

In [21]:
# check missing values in categorical variables in X_train

X_train[categorical].isnull().sum()

Date           0
Location       0
WindGustDir    0
WindDir9am     0
WindDir3pm     0
RainToday      0
dtype: int64

In [22]:
# check missing values in categorical variables in X_test

X_test[categorical].isnull().sum()

Date           0
Location       0
WindGustDir    0
WindDir9am     0
WindDir3pm     0
RainToday      0
dtype: int64

In [23]:
# check missing values in X_train

X_train.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
dtype: int64

In [24]:
# check missing values in X_test

X_test.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
dtype: int64

In [25]:
#You can ignore the warnings, the code still works
def max_value(df3, variable, top):
    return np.where(df3[variable]>top, top, df3[variable])

for df3 in [X_train, X_test]:
    df3['Rainfall'] = max_value(df3, 'Rainfall', 3.2)
    df3['Evaporation'] = max_value(df3, 'Evaporation', 21.8)
    df3['WindSpeed9am'] = max_value(df3, 'WindSpeed9am', 55)
    df3['WindSpeed3pm'] = max_value(df3, 'WindSpeed3pm', 57)

C:\Users\bacch\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\bacch\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\bacch\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

In [26]:
X_train.Rainfall.max(), X_test.Rainfall.max()

(3.2, 3.2)

In [27]:
X_train.Evaporation.max(), X_test.Evaporation.max()

(21.8, 21.8)

In [28]:
X_train.WindSpeed9am.max(), X_test.WindSpeed9am.max()

(55.0, 55.0)

In [29]:
X_train.WindSpeed3pm.max(), X_test.WindSpeed3pm.max()

(57.0, 57.0)

In [30]:
X_train[numerical].describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000,113754.000000
mean,12.193497,23.237216,0.675080,5.151606,8.041154,39.884074,13.978155,18.614756,68.867486,51.509547,1017.640649,1015.241101,4.651801,4.703588,16.995062,21.688643
std,6.388279,7.094149,1.183837,2.823707,2.769480,13.116959,8.806558,8.685862,18.935587,20.530723,6.738680,6.675168,2.292726,2.117847,6.463772,6.855649
min,-8.200000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-7.200000,-5.400000
25%,7.600000,18.000000,0.000000,4.000000,8.200000,31.000000,7.000000,13.000000,57.000000,37.000000,1013.500000,1011.000000,3.000000,4.000000,12.300000,16.700000
50%,12.000000,22.600000,0.000000,4.800000,8.500000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.600000,1015.200000,5.000000,5.000000,16.700000,21.100000
75%,16.800000,28.200000,0.600000,5.400000,8.700000,46.000000,19.000000,24.000000,83.000000,65.000000,1021.800000,1019.400000,6.000000,6.000000,21.500000,26.300000
max,33.900000,48.100000,3.200000,21.800000,14.500000,135.000000,55.000000,57.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,8.000000,40.200000,46.700000


In [31]:
#Might have to install category_encoders
import category_encoders as ce

encoder = ce.BinaryEncoder(cols=['RainToday'])

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

In [32]:
X_train = pd.concat([X_train[numerical], X_train[['RainToday_0', 'RainToday_1']],
                     pd.get_dummies(X_train.Location), 
                     pd.get_dummies(X_train.WindGustDir),
                     pd.get_dummies(X_train.WindDir9am),
                     pd.get_dummies(X_train.WindDir3pm)], axis=1)

In [33]:
X_test = pd.concat([X_test[numerical], X_test[['RainToday_0', 'RainToday_1']],
                     pd.get_dummies(X_test.Location), 
                     pd.get_dummies(X_test.WindGustDir),
                     pd.get_dummies(X_test.WindDir9am),
                     pd.get_dummies(X_test.WindDir3pm)], axis=1)

In [34]:
cols = X_train.columns

In [35]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [36]:
X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])

In [37]:
# y_train.isnull().sum()

In [38]:
#Split X_train and y_train into 1000, and 10000 so we can run them
X_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)

X_train_104 = X_train.sample(1000)
y_train_104 = y_train[X_train_104.index]
print(y_train_104.isnull().sum())
print (X_train_104)
print (y_train_104)

X_train_105 = X_train.sample(10000)
y_train_105 = y_train[X_train_105.index]

0
         MinTemp   MaxTemp Rainfall Evaporation  Sunshine WindGustSpeed  \
83398   0.275534  0.387524   0.0625    0.082569  0.379310      0.139535   
60812   0.494062  0.610586   0.0625    0.091743  0.275862      0.271318   
3187    0.760095  0.705104   0.0000    0.412844  0.675862      0.271318   
98020   0.489311  0.419660   0.6875    0.220183  0.586207      0.581395   
38080   0.334917  0.644612   0.0000    0.275229  0.772414      0.139535   
...          ...       ...      ...         ...       ...           ...   
74653   0.570071  0.487713   0.2500    0.211009  0.165517      0.255814   
14113   0.548694  0.667297   0.0000    0.266055  0.710345      0.341085   
16158   0.494062  0.421550   0.0000    0.082569  0.103448      0.186047   
106175  0.553444  0.493384   0.0000    0.220183  0.586207      0.325581   
52818   0.446556  0.472590   0.0000    0.201835  0.551724      0.410853   

       WindSpeed9am WindSpeed3pm Humidity9am Humidity3pm  ...  NNW   NW    S  \
83398      0.1090

In [39]:
# Training
#Need timer for 3 models we are using
#Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from timeit import default_timer as timer

print("Logistic Regression for size 10^6")
start = timer()
classifier = LogisticRegression(solver='liblinear', random_state = 0)
classifier.fit(X_train, y_train)
end = timer()
print("Training Time: ", end - start, "seconds")

Y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, Y_pred)
print(cm)
print("Accuracy: ", (cm[0,0]+cm[1,1])/cm.sum())

Logistic Regression for size 10^6
Training Time:  1.5181247999999998 seconds
[[20903  1164]
 [ 3105  3267]]
Accuracy:  0.8498892366116952


In [40]:
print("Logistic Regression for size 10^5")
start = timer()
classifier = LogisticRegression(solver='liblinear', random_state = 0)
classifier.fit(X_train_105, y_train_105)
end = timer()
print("Training Time: ", end - start, "seconds")

Y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, Y_pred)
print(cm)
print("Accuracy: ", (cm[0,0]+cm[1,1])/cm.sum())

Logistic Regression for size 10^5
Training Time:  0.08563529999999986 seconds
[[20899  1168]
 [ 3183  3189]]
Accuracy:  0.8470058722177292


In [41]:
print("Logistic Regression for size 10^4")
start = timer()
classifier = LogisticRegression(solver='liblinear', random_state = 0)
classifier.fit(X_train_104, y_train_104)
end = timer()
print("Training Time: ", end - start, "seconds")

Y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, Y_pred)
print(cm)
print("Accuracy: ", (cm[0,0]+cm[1,1])/cm.sum())

Logistic Regression for size 10^4
Training Time:  0.008064599999999977 seconds
[[20367  1700]
 [ 3232  3140]]
Accuracy:  0.8265761805970674


In [42]:
# K_Nearest Neighbors
#Using KNeighborsClassifier Method of neighbors class to use Nearest Neighbor algorithm
from sklearn.neighbors import KNeighborsClassifier
print("K-Nearest Neighbors for size 10^6")
start = timer()
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
end = timer()
print("Training Time: ", end - start, "seconds")

Y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, Y_pred)
print(cm)
print("Accuracy: ", (cm[0,0]+cm[1,1])/cm.sum())

K-Nearest Neighbors for size 10^6
Training Time:  14.147362400000002 seconds
[[20471  1596]
 [ 3792  2580]]
Accuracy:  0.8105418615281831


In [43]:
print("K-Nearest Neighbors for size 10^5")
start = timer()
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train_105, y_train_105)
end = timer()
print("Training Time: ", end - start, "seconds")

Y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, Y_pred)
print(cm)
print("Accuracy: ", (cm[0,0]+cm[1,1])/cm.sum())

K-Nearest Neighbors for size 10^5
Training Time:  0.3526926000000685 seconds
[[20578  1489]
 [ 4240  2132]]
Accuracy:  0.7985512852069342


In [44]:
print("K-Nearest Neighbors for size 10^4")
start = timer()
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train_104, y_train_104)
end = timer()
print("Training Time: ", end - start, "seconds")

Y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, Y_pred)
print(cm)
print("Accuracy: ", (cm[0,0]+cm[1,1])/cm.sum())

K-Nearest Neighbors for size 10^4
Training Time:  0.025194100000021535 seconds
[[20590  1477]
 [ 4758  1614]]
Accuracy:  0.7807588171173389


In [45]:
# Decision Trees 
#Using DecisionTreeClassifier of tree class to use Decision Tree Algorithm
from sklearn.tree import DecisionTreeClassifier
print("Decision Trees for size 10^6")
start = timer()
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
end = timer()
print("Training Time: ", end - start, "seconds")

Y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, Y_pred)
print(cm)
print("Accuracy: ", (cm[0,0]+cm[1,1])/cm.sum())

Decision Trees for size 10^6
Training Time:  3.548577899999941 seconds
[[19026  3041]
 [ 2860  3512]]
Accuracy:  0.7925032525756883


In [46]:
print("Decision Trees for size 10^5")
start = timer()
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train_105, y_train_105)
end = timer()
print("Training Time: ", end - start, "seconds")

Y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, Y_pred)
print(cm)
print("Accuracy: ", (cm[0,0]+cm[1,1])/cm.sum())

Decision Trees for size 10^5
Training Time:  0.18809029999999893 seconds
[[19036  3031]
 [ 3115  3257]]
Accuracy:  0.7838883223742045


In [47]:
print("Decision Trees for size 10^4")
start = timer()
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train_104, y_train_104)
end = timer()
print("Training Time: ", end - start, "seconds")

Y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, Y_pred)
print(cm)
print("Accuracy: ", (cm[0,0]+cm[1,1])/cm.sum())

Decision Trees for size 10^4
Training Time:  0.0163827000000083 seconds
[[18509  3558]
 [ 3059  3313]]
Accuracy:  0.7673265585991068


In [48]:
#End